In [3]:
# Load data from MySQL to perform exploratory data analysis
import settings
import mysql.connector
import pandas as pd
import time

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib inline
# Using plotly.express
import plotly.express as px
import datetime
from IPython.display import clear_output

db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB",
    charset = 'utf8'
)


In [4]:
'''
This plot shows the latest Twitter data within 30 mins and will be automatically updated every minute.
'''
while True:
    clear_output()
    # Load data from MySQL
    timenow = (datetime.datetime.utcnow() - datetime.timedelta(hours=0, minutes=30)).strftime('%Y-%m-%d %H:%M:%S')
    df = pd.read_sql("SELECT id_str, text, created_at, polarity, user_location FROM {} WHERE created_at >= '{}' " \
                     .format(settings.TABLE_NAME, timenow), con=db_connection)
    # UTC for date time at default
    df['created_at'] = pd.to_datetime(df['created_at'])

    # Clean and transform data to enable time series
    result = df.groupby([pd.Grouper(key='created_at', freq='3min'), 'polarity']).count() \
            .unstack(fill_value=0).stack().reset_index()
    result['created_at'] = pd.to_datetime(result['created_at']).apply(lambda x: x.strftime('%m-%d %H:%M'))
    result = result.rename(columns={"id_str": "Num of mentions", "created_at":"Time in UTC"})

    fig = px.line(result, x='Time in UTC', y='Num of mentions', color='polarity')
    fig.show()
    time.sleep(5)

KeyboardInterrupt: 